# Network Science - MDS - UDD 2022
## Tarea 2 Ciencia de Redes


#### Patricio Ramirez
#### Pablo Elgueta




### Redes? Grafos?

Una estructura matemática utilizada para modelar relaciones por pares entre objetos, donde los objetos generalmente se denominan `nodos` y la relación entre ellos `enlaces`.

$G = (V, E)$

$V$ = conjunto de nodos/vértices

$E$ = conjunto de $(x, y)$ enlaces

In [1]:
# Module 1: Comenzando con NetworkX

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import collections

import networkx as nx

%matplotlib inline

import urllib.request as urllib
import io

# path = 'https://saref.github.io/teaching/PyNetworkshop/'

## Trabajemos en una red real

La red de colaboración Arxiv GR-QC (Relatividad General y Cosmología Cuántica) es de e-print arXiv y cubre las colaboraciones científicas entre los artículos de los autores enviados a la categoría de Relatividad General y Cosmología Cuántica. Si un autor $i$ es coautor de un artículo con el autor $j$, el grafo contiene un enlace no dirigido entre $i$ y $j$. Si el artículo es co-autoreado por $k$ autores, esto genera un (sub) grafo completamente conectado con $k$ nodos.

fuente: http://snap.stanford.edu/data/index.html#canets

In [2]:
# crear un grafo de autor a partir del conjunto de datos
import csv
authors_graph = nx.Graph()

with open('data/CA-GrQc.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        authors_graph.add_edge(row[0], row[1])

In [3]:
type(authors_graph)

networkx.classes.graph.Graph

In [4]:
print(authors_graph.number_of_edges())
print(authors_graph.number_of_nodes())

14496
5242


In [5]:
#authors_graph.nodes()

In [6]:
(authors_graph['3466'])

AtlasView({'937': {}, '5233': {}, '8579': {}, '10310': {}, '15931': {}, '17038': {}, '18720': {}, '19607': {}})

In [ ]:
nx.draw(authors_graph,with_labels=True)

#### ¿Podemos encontrar al investigador más influyente/importante en esta red?

##### ¿Cómo evaluamos la importancia de algunas personas en una red?

Dentro de una red social, habrá determinadas personas que desempeñen determinadas funciones importantes. Por ejemplo, puede haber personas hiperconectadas que estén conectadas con muchas, muchas más personas. Ellas serán claves para la difusión de información. Alternativamente, si se tratara de una red de contactos de enfermedades, identificarlos sería útil para detener la propagación de enfermedades. 

#### ¿Cómo se identificaría a estas personas?

### Ejercicio 1

Cree una lista de tuplas (nodo, grado de nodo) y busque el nodo con el grado máximo.

grado de nodo = número de vecinos

In [ ]:
#authors_graph.degree()

In [ ]:
degrees = list(dict(authors_graph.degree()).values())
pd.DataFrame(degrees).transpose()

In [ ]:
kmax = max(degrees)
kmax

In [ ]:
print(f"El nodo que contiene el grado maximo {kmax} es el", list(dict(authors_graph.degree()).keys())[list(dict(authors_graph.degree()).values()).index(kmax)])

El grado de un nodo se traduce en grado de centralidad (que es una versión normalizada de grado)

In [ ]:
nx.degree_centrality(authors_graph)

In [ ]:
print(f"El nodo que contiene el grado de centralidad maximo {max(nx.degree_centrality(authors_graph).values())} es el", list(dict(nx.degree_centrality(authors_graph)).keys())[list(dict(nx.degree_centrality(authors_graph)).values()).index(max(nx.degree_centrality(authors_graph).values()))])

###### Respuesta 1: 

El grado de centralidad para el nodo **21012**, nos permite identificar que pese a ser el **nodo con mayor grado** dentro de la red, en realidad solo presenta enlases con un **1,5%** de los nodos totales de la red.

### Ejercicio 2

Trace un histograma de centralidad de grado de author_graph.

Sugerencia: `plt.hist(list_of_values)` trazará un histograma

(count vs grado)

In [ ]:
plt.hist(list(dict(nx.degree_centrality(authors_graph)).values()), bins=16)
plt.xlabel('Centralidad de Grado')
plt.ylabel('Cantidad de Nodos') 
plt.show()

###### Respuesta 2: 
En primer lugar, es posible apreciar que la mayoría de los nodos presentan una baja Centralidades de Grado inferior a 0,001. 

De acuerdo al gráfico es posible concluir que tenemos una red de **libre escala**.


#### Echemos un vistazo a los componentes conectados de un grafo.

En la teoría de grafos, un componente conectado (o simplemente un componente) de un grafo **no dirigido** es un subgrafo en el que dos vértices cualesquiera están conectados entre sí por caminos, y que no está conectado a ningún vértice adicional en el supergrafo.

In [ ]:
print([len(c) for c in sorted(nx.connected_components(authors_graph),
                              key=len, reverse=True)])


Nota: la función sorted () tiene un parámetro opcional llamado `key` que toma una función como su valor. Esta función `key` transforma cada elemento antes de ordenar, toma el valor y se usa dentro de `sorted` en lugar del valor original.

In [ ]:
# Guardamos subgrafos en una lista
graphs = [authors_graph.subgraph(c).copy() for c in sorted(nx.connected_components(authors_graph), key=len, reverse=True)]


# MUNDO PEQUEÑO
### Ejercicio 3
##### Seis grados de separación, número de Erdos, número de Bacon !!



Encuentre el "número" del autor '22504' del grafo `author_graph`, si no hay conexión entre los nodos, asignele el número '-1'.
También trace un histograma del "número" autor '22504'.

Encuentre la longitud de ruta más corta promedio en el primer componente, es decir, `graphs[0]`

SUGERENCIA: `nx.shortest_path_length`

In [ ]:
d = {}
for node in authors_graph.nodes():#para cada nodo
    try:
        #calcula la longitud del camino mas corto entre node y `22504`
        d[node] = nx.shortest_path_length(authors_graph, '22504', node) #21012;22504
    except:
        #si arroja error (no hay camino) asinga un -1.
        d[node] = int(-1) #21012;22504
        next

In [ ]:
d

In [ ]:
plt.hist(list(d.values()))
plt.xlabel('Distancias de Nodo 22504 en la red')
plt.ylabel('Cantidad de Nodos')
plt.show()

###### Respuesta 3.1.

En este gráfico es osible apreciar que una cantidad significativa de nodos no presentan conexión con el nodo 22504, lo cual se puede atribuir al hecho de que aquellos nodos que no presentan conexión con el nodo 22504 puden ser nodos aislados o pertenecientes a otra componente, por lo cual no presentan ninguna conexión con la Primera Componenete Graph[0].

In [ ]:
d = {}
for node in graphs[0].nodes():#para cada nodo
    try:
        #calcula la longitud del camino mas corto entre node y `22504`
        d[node] = nx.shortest_path_length(graphs[0], '22504', node) #21012;22504
    except:
        #si arroja error (no hay camino) asinga un -1.
        d[node] = int(-1) #21012;22504
        next

In [ ]:
d

In [ ]:
plt.hist(list(d.values()), bins=12)
plt.xlabel('Distancias de Nodo 22504 en Primer Componente')
plt.ylabel('Cantidad de Nodos')
plt.show()

In [ ]:
# Calcule el promedio de la longitud de los caminos más cortos de todo el grafo aca:

np.mean(list(d.values()))

###### Respuesta 3.2.

En este caso, acotando el análisis solo a la Prmera Componente Graph[o] se aprecia que el nodoposee una distancia de entre 1 a 12 con el resto de nodos de la Componente y que la mayoría se encuentran entre 4 y 8, lo que permite apreciar una distribución normal en las distancias para el nodo 22504, lo cual se confirma en que la longitud mas corta promedio es 5,6 un valor muy cercano a la mitad del rango de distancias para dicho nodo.